In [1]:
!pip install qiskit
!pip install qiskit-aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 104.0 MB/s eta 0:00:00


In [2]:
# Deutsch Algorithm using Qiskit 2.x
# Compatible with Qiskit 2.0+ (2024–2025)

from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Statevector
from qiskit_aer import AerSimulator

# Choose the function type: 'constant_0', 'constant_1', 'balanced_0', 'balanced_1'
function_type = 'balanced_1'

def deutsch_oracle(qc, function_type):
    """Implements oracle Uf for given function type."""
    if function_type == 'constant_0':
        # f(x)=0 → Do nothing
        pass
    elif function_type == 'constant_1':
        # f(x)=1 → Apply X on the output qubit
        qc.x(1)
    elif function_type == 'balanced_0':
        # f(x)=x → Apply CNOT (control: input, target: output)
        qc.cx(0, 1)
    elif function_type == 'balanced_1':
        # f(x)=NOT(x) → Apply X, then CNOT, then X
        qc.x(0)
        qc.cx(0, 1)
        qc.x(0)

In [3]:
# Step 1: Initialize quantum circuit with 2 qubits and 1 classical bit
qc = QuantumCircuit(2, 1)

# Step 2: Initialize |x>|y> = |0>|1> and apply Hadamard
qc.x(1)             # Set output qubit to |1>
qc.barrier()
qc.h([0, 1])        # Apply Hadamard to both qubits

# Step 3: Apply the oracle
qc.barrier()
deutsch_oracle(qc, function_type)

In [4]:
# Step 4: Apply Hadamard to input qubit
qc.barrier()
qc.h(0)

# Step 5: Measure the first qubit
qc.measure(0, 0)

# Visualize circuit
print(qc.draw(output="text"))

           ░ ┌───┐ ░ ┌───┐     ┌───┐ ░ ┌───┐┌─┐
q_0: ──────░─┤ H ├─░─┤ X ├──■──┤ X ├─░─┤ H ├┤M├
     ┌───┐ ░ ├───┤ ░ └───┘┌─┴─┐└───┘ ░ └───┘└╥┘
q_1: ┤ X ├─░─┤ H ├─░──────┤ X ├──────░───────╫─
     └───┘ ░ └───┘ ░      └───┘      ░       ║ 
c: 1/════════════════════════════════════════╩═
                                             0 


In [5]:
# Step 6: Simulate
sim = AerSimulator()
qc_compiled = transpile(qc, sim)
result = sim.run(qc_compiled).result()
counts = result.get_counts()

print("\nMeasurement results:", counts)

# Interpret result
if list(counts.keys())[0] == '0':
    print("→ Function is CONSTANT.")
else:
    print("→ Function is BALANCED.")


Measurement results: {'1': 1024}
→ Function is BALANCED.


In [6]:
# Qiskit_Deutsch_and_Tasks.py
# Run in a Jupyter notebook with Qiskit 2.x (2024-2025). Install with:
# pip install qiskit qiskit-aer

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_bloch_multivector, plot_histogram
from qiskit_aer.noise import NoiseModel, depolarizing_error
import matplotlib.pyplot as plt
import numpy as np

# -------------------------
# USER-SWITCHABLE VARIABLE
# -------------------------
# For single-bit Deutsch: choose one of:
# 'constant_0', 'constant_1', 'balanced_0', 'balanced_1', 'custom_x_xor_1'
function_type = 'balanced_1'   # <-- change here to test different oracles

# -------------------------
# Helper: Single-bit oracle (Uf on |x>|y> -> |x>|y XOR f(x)>)
# -------------------------
def single_bit_oracle(qc: QuantumCircuit, x_qubit: int, y_qubit: int, f_type: str):
    """
    Implements Uf for single-bit f.
    f_type: 'constant_0', 'constant_1', 'balanced_0', 'balanced_1', 'custom_x_xor_1'
    For balanced_0: f(0)=0, f(1)=1 (i.e. f(x)=x)  -> apply CNOT(x->y)
    For balanced_1: f(0)=1, f(1)=0 (i.e. f(x)=x XOR 1) -> apply CNOT + X on target or apply X on target then CNOT then X (equivalently)
    constant_0: do nothing
    constant_1: flip target (X on y)
    custom_x_xor_1: same as balanced_1 (demonstration)
    """
    if f_type == 'constant_0':
        # f(x) = 0: do nothing
        pass
    elif f_type == 'constant_1':
        # f(x) = 1: flip y (X on y) regardless of x
        qc.x(y_qubit)
    elif f_type == 'balanced_0':
        # f(x) = x  -> y ^= x
        qc.cx(x_qubit, y_qubit)
    elif f_type == 'balanced_1':
        # f(x) = x XOR 1 -> y ^= x and then flip (equiv: flip y before and after CNOT or flip after as required)
        # We'll implement: X on y, CX, X on y (so target toggled based on x XOR 1)
        qc.x(y_qubit)
        qc.cx(x_qubit, y_qubit)
        qc.x(y_qubit)
    elif f_type == 'custom_x_xor_1':
        qc.x(y_qubit)
        qc.cx(x_qubit, y_qubit)
        qc.x(y_qubit)
    else:
        raise ValueError("Unknown f_type")

# -------------------------
# Deutsch algorithm (single-bit) implementation
# -------------------------
def deutsch_single_shot(f_type='balanced_1', shots=1024, show_circuit=False, show_bloch=True):
    # Qubits: q0 = input |x>, q1 = output ancilla |y>
    qc = QuantumCircuit(2, 1)
    # initialize ancilla to |1>
    qc.x(1)
    # create superposition on input and put ancilla into |-> if desired: H on both
    qc.h(0)
    qc.h(1)
    # optionally visualize Bloch before oracle
    if show_bloch:
        sv_before = Statevector.from_instruction(qc)
        print("Bloch vectors BEFORE oracle (showing for qubit-0 and qubit-1):")
        _ = plot_bloch_multivector(sv_before)
        plt.show()

    # apply oracle Uf
    single_bit_oracle(qc, x_qubit=0, y_qubit=1, f_type=f_type)

    # optionally visualize Bloch after oracle
    if show_bloch:
        sv_after_oracle = Statevector.from_instruction(qc)
        print("Bloch vectors AFTER oracle (showing for qubit-0 and qubit-1):")
        _ = plot_bloch_multivector(sv_after_oracle)
        plt.show()

    # apply final H on input qubit and measure
    qc.h(0)
    qc.measure(0, 0)

    if show_circuit:
        print(qc.draw(fold=-1))

    # simulate
    backend = AerSimulator()
    t_qc = transpile(qc, backend)
    job = backend.run(t_qc, shots=shots)
    result = job.result()
    counts = result.get_counts()
    # interpret: measurement 0 => constant, 1 => balanced
    outcome = 0 if '0' in counts and counts.get('0', 0) > counts.get('1', 0) else 1
    measured_str = max(counts, key=counts.get)
    classification = "CONSTANT" if measured_str[0] == '0' else "BALANCED"
    print(f"\nFunction type: {f_type}")
    print("Measurement counts:", counts)
    print("Inferred measurement (majority):", measured_str)
    print("Conclusion:", classification)
    return qc, counts

# -------------------------
# Task: Custom Oracle f(x) = x XOR 1 detection (we included as custom_x_xor_1)
# -------------------------
# This is covered by calling deutsch_single_shot(function_type='custom_x_xor_1')


# -------------------------
# Deutsch-Jozsa Extension: n=2 qubits (input register size = 2)
# Distinguish constant vs balanced for 2-bit input functions.
# We construct canonical balanced and constant oracles.
# -------------------------
def dj_oracle_2qubit(qc: QuantumCircuit, input_qubits: list, ancilla_qubit: int, f_type='constant_0'):
    """
    Implements a 2-qubit-to-1 oracle Uf(|x1,x0>|y>) -> |x1,x0>|y XOR f(x1,x0)>
    f_type choices:
        'const_0', 'const_1' -> constant functions
        'balanced_parity' -> f(x)=x0 XOR x1 (balanced)
        'balanced_half'   -> a balanced function that returns 1 for two of the inputs (e.g. f(00)=0,f(01)=1,f(10)=1,f(11)=0)
    You can add more patterns as needed.
    """
    if f_type == 'const_0':
        pass
    elif f_type == 'const_1':
        qc.x(ancilla_qubit)
    elif f_type == 'balanced_parity':
        # f(x) = x0 XOR x1 -> apply CX from q0 and q1 (two CX gates)
        # One way: apply CX(input0 -> ancilla) and CX(input1 -> ancilla)
        qc.cx(input_qubits[0], ancilla_qubit)
        qc.cx(input_qubits[1], ancilla_qubit)
    elif f_type == 'balanced_half':
        # Example mapping: f(00)=0, f(01)=1, f(10)=1, f(11)=0
        # Implement by controlled flips on ancilla where appropriate:
        # Flip ancilla for inputs 01 and 10 (i.e. when exactly one of the inputs is 1).
        # This is essentially parity again; using parity example as balanced.
        qc.cx(input_qubits[0], ancilla_qubit)
        qc.cx(input_qubits[1], ancilla_qubit)
    else:
        raise ValueError("Unknown f_type for 2-qubit DJ")

def deutsch_jozsa_2qubit(f_type='balanced_parity', shots=1024, show_circuit=False):
    # Input reg size 2, ancilla 1
    qc = QuantumCircuit(3, 2)
    # ancilla into |1>
    qc.x(2)
    # H on all input and ancilla as per DJ
    qc.h(0); qc.h(1); qc.h(2)

    # oracle
    dj_oracle_2qubit(qc, [0,1], ancilla_qubit=2, f_type=f_type)

    # final Hadamards on input only and measure inputs
    qc.h(0); qc.h(1)
    qc.measure([0,1], [0,1])

    if show_circuit:
        print(qc.draw(fold=-1))

    backend = AerSimulator()
    t_qc = transpile(qc, backend)
    result = backend.run(t_qc, shots=shots).result()
    counts = result.get_counts()
    print("\nDeutsch–Jozsa (2-qubit input)")
    print("Function type (oracle):", f_type)
    print("Measurement counts (input register):", counts)
    # in DJ: result 00 means constant; any non-zero result indicates balanced
    most = max(counts, key=counts.get)
    classification = "CONSTANT" if most == '00' else "BALANCED"
    print("Conclusion:", classification)
    return qc, counts

# -------------------------
# Noise model experiment: simple depolarizing noise applied to single- and two-qubit gates
# -------------------------
def create_simple_noise_model(p1=0.001, p2=0.01):
    noise_model = NoiseModel()
    # depolarizing on single-qubit gates
    error1 = depolarizing_error(p1, 1)
    # depolarizing on two-qubit gates
    error2 = depolarizing_error(p2, 2)
    # apply to common gates
    noise_model.add_all_qubit_quantum_error(error1, ['h', 'x'])
    noise_model.add_all_qubit_quantum_error(error2, ['cx'])
    return noise_model

def run_deutsch_with_noise(f_type='balanced_1', shots=2048, noise_p1=0.002, noise_p2=0.02):
    qc = QuantumCircuit(2, 1)
    qc.x(1); qc.h(0); qc.h(1)
    single_bit_oracle(qc, 0, 1, f_type)
    qc.h(0); qc.measure(0, 0)
    backend = AerSimulator()
    noise = create_simple_noise_model(noise_p1, noise_p2)
    t_qc = transpile(qc, backend)
    job = backend.run(t_qc, shots=shots, noise_model=noise)
    result = job.result()
    counts = result.get_counts()
    print("\nDeutsch with noise (p1={}, p2={})".format(noise_p1, noise_p2))
    print("Oracle:", f_type)
    print("Counts:", counts)
    plot_histogram(counts)
    plt.show()
    return counts

# -------------------------
# Classical evaluator (simple): query f(x) sequentially and count queries until classification
# For single-bit: worst-case needs 2 queries (query f(0) and f(1))
# For n-bit (Deutsch–Jozsa), worst-case classical queries to be certain = 2^(n-1) + 1
# (We will demonstrate for n=1 and n=2)
# -------------------------
def classical_query_single(f_function):
    """Queries f(0) then f(1) until it can decide; returns number of queries and decision."""
    v0 = f_function(0)
    # If we want a decision after one query: we cannot decide, so must query second
    v1 = f_function(1)
    queries = 2
    conclusion = "CONSTANT" if v0 == v1 else "BALANCED"
    return queries, conclusion

def classical_query_2bit(f_function):
    """
    Sequentially queries inputs until decision is forced.
    Worst-case expected to need 3 queries (2^(n-1) + 1 = 3 for n=2).
    We'll query distinct inputs until we can conclude.
    """
    queried = {}
    inputs = [(0,0), (0,1), (1,0), (1,1)]
    for i, inp in enumerate(inputs):
        val = f_function(inp)
        queried[inp] = val
        # Check if all currently seen values are the same AND we've seen > 2 inputs? Only then we can conclude constant.
        vals = list(queried.values())
        if len(vals) >= 3:
            # if all three same -> must be constant (can't be balanced because balanced has 2 zeros and 2 ones)
            if all(v == vals[0] for v in vals):
                return len(vals), "CONSTANT"
        # If we see both 0 and 1 among queried, then it's BALANCED
        if 0 in vals and 1 in vals:
            return len(vals), "BALANCED"
    # if we exhausted all four, deduce
    final = "CONSTANT" if len(set(queried.values())) == 1 else "BALANCED"
    return 4, final

# -------------------------
# Demo runs
# -------------------------

# 1) Deutsch single-bit run and custom oracle
qc_single, counts_single = deutsch_single_shot(function_type, shots=1024, show_circuit=True, show_bloch=False)

# 2) Custom oracle f(x)=x XOR 1 detection:
print("\nCustom oracle test (f(x) = x XOR 1):")
qc_custom, counts_custom = deutsch_single_shot('custom_x_xor_1', shots=1024, show_circuit=False, show_bloch=False)

# 3) Bloch visualization demo specifically after H and after oracle for a chosen function
# (re-run with show_bloch=True if you want visuals - earlier call showed circuits; use below to see Bloch)
_ = deutsch_single_shot(function_type, shots=256, show_circuit=False, show_bloch=True)

# 4) Deutsch–Jozsa 2-qubit
qc_dj, counts_dj = deutsch_jozsa_2qubit(f_type='balanced_parity', shots=1024, show_circuit=True)

# 5) Noise impact
counts_noise = run_deutsch_with_noise(function_type, shots=2048, noise_p1=0.002, noise_p2=0.02)

# 6) Classical vs Quantum queries (single-bit)
# Define the same f as Python-callable
def f_single_python(x):
    mapping = {
        'constant_0': {0:0, 1:0},
        'constant_1': {0:1, 1:1},
        'balanced_0': {0:0, 1:1},
        'balanced_1': {0:1, 1:0},
        'custom_x_xor_1': {0:1, 1:0}
    }
    return mapping[function_type][x]

queries_needed, classical_decision = classical_query_single(f_single_python)
print(f"\nClassical (single-bit) queries needed (simple sequential): {queries_needed}, decision: {classical_decision}")
print("Quantum Deutsch: 1 quantum query -> decides CONSTANT vs BALANCED (up to measurement noise).")

# 7) Classical vs Quantum (2-qubit DJ)
# Example: balanced_parity f(x) = parity; implement python callable
def f_2bit_python(inp):
    if inp == (0,0): return 0
    if inp == (0,1): return 1
    if inp == (1,0): return 1
    if inp == (1,1): return 0

q_needed_2bit, cls_decision_2bit = classical_query_2bit(f_2bit_python)
print(f"\nClassical (2-bit) worst-case queries (sample run) required: {q_needed_2bit}, decision on sampled queries: {cls_decision_2bit}")
print("Quantum Deutsch–Jozsa for n=2: 1 quantum query is sufficient in the ideal (noise-free) circuit to decide CONSTANT vs BALANCED.\n")
print("Note: Classical worst-case needed to be CERTAIN = 2^(n-1) + 1 (for n=2 -> 3 queries).")

# End of notebook cell

     ┌───┐               ┌───┐┌─┐
q_0: ┤ H ├────────────■──┤ H ├┤M├
     ├───┤┌───┐┌───┐┌─┴─┐├───┤└╥┘
q_1: ┤ X ├┤ H ├┤ X ├┤ X ├┤ X ├─╫─
     └───┘└───┘└───┘└───┘└───┘ ║ 
c: 1/══════════════════════════╩═
                               0 

Function type: balanced_1
Measurement counts: {'1': 1024}
Inferred measurement (majority): 1
Conclusion: BALANCED

Custom oracle test (f(x) = x XOR 1):

Function type: custom_x_xor_1
Measurement counts: {'1': 1024}
Inferred measurement (majority): 1
Conclusion: BALANCED
Bloch vectors BEFORE oracle (showing for qubit-0 and qubit-1):
Bloch vectors AFTER oracle (showing for qubit-0 and qubit-1):

Function type: balanced_1
Measurement counts: {'1': 256}
Inferred measurement (majority): 1
Conclusion: BALANCED
     ┌───┐          ┌───┐     ┌─┐   
q_0: ┤ H ├───────■──┤ H ├─────┤M├───
     ├───┤       │  └───┘┌───┐└╥┘┌─┐
q_1: ┤ H ├───────┼────■──┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐┌─┴─┐└───┘ ║ └╥┘
q_2: ┤ X ├┤ H ├┤ X ├┤ X ├──────╫──╫─
     └───┘└───┘└───┘└───┘    